In [1]:

#
# The environment
#

import requests
import urllib3

# where data is located
datadir = '../snowdata/'

# environment variables
# anything of interest in this file should be of the format
#     export A=B
envfile = '.setenv.sh'

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

headers = {"Content-Type":"application/json","Accept":"application/json"}



In [2]:
#
# Find environment variables
#
envvars = {}
with open( envfile) as env:
    for line in env:
        parts = line.split()
        if len(parts) >= 2 and not parts[0].startswith('#'):
            name = parts[1].split('=')[0]
            valParts = parts[len(parts)-1].split('=')
            value = valParts[len(valParts)-1]
            envvars[name]= value.strip('""')
                
user = envvars['SN_USER']
passw = envvars['SN_PASS']
urlbase = envvars['SN_URL']
proxy = envvars['SN_PROXY']
proxies = { 'https' : proxy, 'http' : proxy }

In [3]:
#
#Test get
#

url = urlbase + '/api/now/table/incident?sysparm_limit=1'
response = requests.get(url, auth=(user, passw), headers=headers, proxies = proxies, verify = False )

if response.status_code != 200: 
    print('Status:', response.status_code, 'Headers:', response.headers, 'Error Response:', response.json())
else:
    test_data = response.json()
    print(test_data['result'][0]['number'])


INC0000060


In [4]:
#
#Test post
#

import datetime

now = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

url = urlbase + '/api/now/table/incident'
post_data = { "short_description" : "test-posting " + now }
response = requests.post(url, auth=(user, passw), headers=headers, proxies = proxies, verify = False, json = post_data )

if response.status_code != 201: 
    print('Status:', response.status_code, 'Headers:', response.headers, 'Error Response:', response.json())
else:
    post_test_data = response.json()
    print(post_test_data['result']['number'])


INC0010018


In [7]:
#
# Add IP router
#

import datetime

now = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
name = "testrouter-"+now

url = urlbase + '/api/now/cmdb/instance/cmdb_ci_ip_router'
post_data = { "source": "ServiceNow", "attributes": { "name": name } } 

response = requests.post(url, auth=(user, passw), headers=headers, proxies = proxies, verify = False, json = post_data )

if response.status_code != 201: 
    print('Status:', response.status_code, 'Headers:', response.headers, 'Error Response:', response.json())
else:
    response_json = response.json()
    print(response_json['result']['attributes']['name'])
    


testrouter-2020-10-01-12-40-48


In [11]:
#
# get router
#

name = 'testrouter-2020-10-01-12-40-48'
url = urlbase + '/api/now/cmdb/instance/cmdb_ci_ip_router?sysparm_query=name%3D'+name
response = requests.get(url, auth=(user, passw), headers=headers, proxies = proxies, verify = False )

if response.status_code != 200: 
    print('Status:', response.status_code, 'Headers:', response.headers, 'Error Response:', response.json())
else:
    test_data = response.json()
    result = test_data['result'][0]
    print(result['sys_id'], '=', result['name'])
    

7df52e9e07a31010e3c2f7d08c1ed09a = testrouter-2020-10-01-12-40-48


In [21]:
#
# get location
#

name = '100 South Charles Street, Baltimore,MD'
url = urlbase + '/api/now/table/cmn_location?sysparm_query=name%3D'+name
response = requests.get(url, auth=(user, passw), headers=headers, proxies = proxies, verify = False )

if response.status_code != 200: 
    print('Status:', response.status_code, 'Headers:', response.headers, 'Error Response:', response.json())
else:
    test_data = response.json()
    result = test_data['result'][0]
    print(result['sys_id'], '=', result['name'])
    

25ab9c4d0a0a0bb300f7dabdc0ca7c1c = 100 South Charles Street, Baltimore,MD


In [48]:
#
# Finds mapping for a given data file
#
from pathlib import Path

def find_mapping ( datafile ):
    mapfile = datadir + Path(datafile).stem.strip('.') + '.map'
    mapping = {}
    with open(mapfile) as mf:
        for line in mf:
            parts = line.split()
            if ( len(parts) == 2 ):
                mapping[parts[0]] = parts[1]
    return mapping
        

In [69]:
#
# Load the IP Complex data
#

import pandas as pa

datafile = 'ip_complex.dsv'
file = datadir + datafile

df = pa.read_csv(file, sep='|')
mapping = find_mapping(datafile)

url = urlbase + '/api/now/table/cmn_location'

for index, row in df.iterrows():
    attributes = {}
    for f in mapping:
        name = mapping[f]
        value =  row[f]
        attributes[name] = value

    post_data = attributes
    post_data['source'] = 'ServiceNow'
    #print(post_data)
    response = requests.post(url, auth=(user, passw), headers=headers, proxies = proxies, verify = False, json = post_data )
    if response.status_code != 201: 
        print('Status:', response.status_code, 'Headers:', response.headers, 'Error Response:', response.json())
    else:
        response_json = response.json()
        #print(response_json)
        print(response_json['result']['sys_id'])



39e958ea07271010e3c2f7d08c1ed04a
8ae9946207671010e3c2f7d08c1ed017
